# Pipeline de Treinamento do Cerebra
Este notebook executa todas as etapas do projeto Cerebra em sequência.


## 0. Preparação do ambiente

### Clonar o repositório do projeto

In [ ]:
!git clone https://github.com/cadusouza2001/cerebra.git
%cd cerebra

### Instalação de dependências

In [ ]:
!pip install aiohttp beautifulsoup4 txtai torch google-generativeai

### (Opcional) Montar o Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Configuração de variáveis de ambiente

In [ ]:
from google.colab import userdata
import os
from google.colab.userdata import SecretNotFoundError

possible_keys = ['GOOGLE_API_KEY'] + [f'GOOGLE_API_KEY_{i}' for i in range(1, 10)]
api_keys = []
for k in possible_keys:
    try:
        value = userdata.get(k)
        if value:
            api_keys.append(value)
    except SecretNotFoundError:
        continue

if not api_keys:
    raise RuntimeError('Nenhuma chave de API encontrada. Defina pelo menos GOOGLE_API_KEY nos secrets do Colab.')

os.environ['GEMINI_API_KEYS'] = ','.join(api_keys)

# Outros caminhos configuráveis
os.environ['DATASET_FILE'] = 'qa_dataset/spark_qa_generative_dataset.jsonl'
os.environ['OUTPUT_MODEL_DIR'] = 'spark_expert_model'

## 1. Aquisição de dados

### 1.1 Raspagem da documentação

In [ ]:
!python src/scrape_fast_resumable.py

### 1.2 Criação do índice semântico

In [ ]:
!python src/index_spark_docs.py

### 1.3 Geração do dataset

In [ ]:
!python src/generate_qa_dataset.py

## 2. Treinamento

### 2.1 Treinamento do modelo seq2seq

In [ ]:
!python src/train.py

### 2.2 Fine-tuning de modelo pré-treinado (T5)

In [ ]:
!python src/fine_tune_t5.py

## 3. Validação

### 3.1 Avaliação do modelo seq2seq

In [ ]:
!python src/evaluate_model.py

### 3.2 Cálculo de métricas (BLEU)

In [ ]:
import json
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def compute_bleu(path='evaluation_results.jsonl'):
    data=[json.loads(l) for l in open(path,'r',encoding='utf-8')]
    smooth=SmoothingFunction().method1
    scores=[sentence_bleu([d['expected'].split()], d['prediction'].split(), smoothing_function=smooth) for d in data]
    print('BLEU:', sum(scores)/len(scores))

compute_bleu()

### 3.3 Geração de gráficos

In [ ]:
!python src/plot_graphs.py

## 4. Perguntas interativas

### Perguntas usando o modelo seq2seq

In [ ]:
!python src/run_qa_system.py

### Perguntas usando o modelo T5

In [ ]:
!python src/run_qa_t5.py 'What is Apache Spark?'